In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,auc

In [173]:
Classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    SVC(kernel ='linear'),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier(),
    GaussianNB()]

In [174]:
data2=pd.read_csv("test_with_solutions.csv")
data1=pd.read_csv("train.csv") #currently in use
#data3=pd.read_csv("agr_en_train.csv")
#data4=pd.read_csv("agr_en_tw_gold.csv")

In [175]:
data1.columns = ['Insult','Date','Comment']
df1=pd.DataFrame([data1['Comment'],data1['Insult']]).T
data2.columns = ['Insult','Date','Comment', 'Usage']
df2=pd.DataFrame([data2['Comment'],data2['Insult']]).T

In [176]:
frames=[df1['Comment'],df2['Comment']]
result=pd.DataFrame(pd.concat(frames))
result.shape

(6594, 1)

In [177]:
feature_extraction = TfidfVectorizer()
data = feature_extraction.fit_transform(result["Comment"])
data.shape

(6594, 22740)

In [178]:
X_train=data[0:3947,:]
X_test=data[3947:,:]
#df1['Insult']=df1['Insult'].astype(int)
#df2['Insult']=df2['Insult'].astype(int)
#y_train['Insult']=df1['Insult']
#y_test['Insult']=df2['Insult']
X_test.shape
X_train.shape

(3947, 22740)

In [179]:
print(X_train[0])

  (0, 22632)	0.1799461521484288
  (0, 8803)	0.6425490633022721
  (0, 22644)	0.26847292798247196
  (0, 4814)	0.6947462634061918


In [180]:
X_train.shape
count=0
for a in df2['Insult']:
    if a==1:
        count=count+1
count

693

In [181]:
#y_test['Insult']=df2['Insult']
y_train=df1['Insult']
y_train=pd.DataFrame(y_train)
y_test=df2['Insult']
y_test=pd.DataFrame(y_test)
y_train.shape
y_train=y_train.astype(int)
y_test=y_test.astype(int)
#df2['Insult']=df2['Insult'].astype(int)

In [182]:
y_train.shape

(3947, 1)

In [183]:

data3=pd.read_csv("kmeans_data.csv")
X=pd.DataFrame([data3['X1'],data3['X2']]).T
X=data3.values
#print(X_train)
X_train=X[:251,0:2]
Y_train_sol=X[:251,2:3]

X_test=X[251:299,0:2]
Y_test_sol=X[251:299,2:3]
#print(Y_test_sol)
print(X_train.shape)


(251, 2)


In [184]:
class K_Means:
    def __init__(self, k=2, tol=0.001, max_iter=10):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[1:2]
            
        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []
            
            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                #distances = distance.euclidean(featureset,self.centroids[0]) 
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break

    def predict(self,data_test):
        distances = [np.linalg.norm(data_test-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification




In [185]:
clf = K_Means()
clf.fit(X_train)
pred=[]
for i in X_test:
    pred.append(clf.predict(i))
print(len(pred))

48


C:\Users\Taher_Makda\Anaconda3\lib\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Taher_Makda\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [186]:
print("The AUC score is:")
roc_auc_score(Y_test_sol, pred)
#metrics.roc_curve(y_test, pred)
# fpr, tpr, thresholds = metrics.roc_curve(y_test,pred,pos_label=2)
# tpr.shape
# auc = metrics.auc(fpr,tpr)
# #print('Auc of '+classifier.__class__.__name__+'is '+str(auc))
# auc

The AUC score is:


0.5